# Day 23 — "Evaluation Metrics for Segmentation (IoU, mIoU, F1, Boundary Accuracy)"

Losses train models, metrics judge them. Different metrics reward different geometric properties: overlap, balance, and boundary sharpness.


In [1]:
# Ensure repo root is on sys.path for local imports
import sys
from pathlib import Path

repo_root = Path.cwd()
if not (repo_root / "days").exists():
    for parent in Path.cwd().resolve().parents:
        if (parent / "days").exists():
            repo_root = parent
            break

sys.path.insert(0, str(repo_root))
print(f"Using repo root: {repo_root}")


Using repo root: /media/abdul-aziz/sdb7/masters_research/math_course_dlcv


## 1. Core Intuition

- Metrics answer *what good segmentation means*.
- IoU rewards overlap, F1 balances precision/recall, boundary metrics reward crisp edges.
- The "best" mask depends on the metric you care about.


## 2. Confusion Matrix for Segmentation

| | Pred FG | Pred BG |
| --- | --- | --- |
| GT FG | TP | FN |
| GT BG | FP | TN |

Every segmentation metric is a function of these four counts.


## 3. IoU (Intersection over Union)

IoU = TP / (TP + FP + FN)

- Strict overlap measure.
- Ignores true negatives, so it is robust to imbalance.


## 4. mIoU (Mean IoU)

mIoU averages IoU across classes so rare classes are not dominated by large ones.


## 5. Precision, Recall, F1

Precision = TP / (TP + FP)

Recall = TP / (TP + FN)

F1 balances the two, and is identical to Dice for binary masks.


## 6. Dice (F1 for Segmentation)

Dice = 2TP / (2TP + FP + FN)

Dice emphasizes foreground overlap and is common in medical segmentation.


## 7. Boundary Accuracy (Boundary F1)

Boundary metrics score contour alignment rather than area overlap.
They are essential for thin objects or crisp outlines.


## 8. Python — Metric Implementations (NumPy)

`days/day23/code/metrics.py` provides IoU, Dice, F1, mean IoU, and boundary F1.


In [2]:
import numpy as np
from days.day23.code.metrics import (
    iou_score,
    dice_score,
    precision_recall_f1,
    boundary_f1,
)

rng = np.random.default_rng(0)
gt = (rng.random((64, 64)) > 0.9).astype(np.float32)
pred = rng.random((64, 64)).astype(np.float32)

print("IoU:", iou_score(pred, gt))
print("Dice:", dice_score(pred, gt))
precision, recall, f1 = precision_recall_f1(pred, gt)
print("Precision:", precision)
print("Recall:", recall)
print("F1:", f1)
print("Boundary F1:", boundary_f1(pred, gt))


IoU: 0.09034267953255778
Dice: 0.1657142860548105
Precision: 0.09825750285660334
Recall: 0.5286458345608182
F1: 0.1657140220578973
Boundary F1: 0.9434053915154587


## 9. Visualization — Metrics vs Threshold

`days/day23/code/visualizations.py` plots metric curves across thresholds on a synthetic mask.


In [3]:
from days.day23.code.visualizations import plot_metric_curves

RUN_FIGURES = False

if RUN_FIGURES:
    plot_metric_curves()
else:
    print("Set RUN_FIGURES = True to regenerate Day 23 figures inside days/day23/outputs/.")


Set RUN_FIGURES = True to regenerate Day 23 figures inside days/day23/outputs/.


## 10. Metric Geometry — What Each Metric Rewards

| Metric | Rewards | Penalizes |
| --- | --- | --- |
| Accuracy | Majority correctness | Minority neglect |
| IoU | Overlap quality | Small shifts |
| mIoU | Class balance | Rare-class noise |
| Precision | False alarms | Missed objects |
| Recall | Missed objects | Over-prediction |
| F1 / Dice | Balance | Extreme bias |
| Boundary F1 | Edge accuracy | Boundary blur |


## 11. Metric Selection by Task

| Task | Recommended Metrics |
| --- | --- |
| Binary segmentation | IoU + Dice |
| Multi-class segmentation | mIoU |
| Medical imaging | Dice + Boundary F1 |
| Remote sensing | IoU + F1 |
| Roads / rivers | Boundary F1 |
| Change detection | Precision + Recall + F1 |
| Benchmark reporting | mIoU |


## 12. Mini Exercises

1. Compute IoU and F1 for the same prediction and compare values.
2. Evaluate a crisp-boundary but underfilled mask vs an overfilled one.
3. Plot IoU vs threshold curve for your dataset.
4. Compute per-class IoU for multi-class masks.
5. Compare Dice vs Boundary F1 on thin road masks.


## 13. Key Takeaways

- Metrics define what success means.
- IoU is the overlap gold standard; mIoU enforces class balance.
- F1/Dice balance precision and recall.
- Boundary metrics capture geometric fidelity.
- Choose metrics that align with the real task.
